In [60]:
from langchain_community.llms import Ollama

llm = Ollama(model = "llama3.1")

In [62]:
llm.invoke("hola")

'¡Hola! ¿En qué puedo ayudarte hoy?'

In [39]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_path = "https://blog.langchain.dev/langgraph/" 
)

docs = loader.load()

In [40]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True
)

In [41]:
all_splits = text_splitter.split_documents(docs)

In [42]:
from langchain_community.embeddings import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="nomic-embed-text"
)

In [43]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding
)

In [44]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)


In [45]:
retriever.get_relevant_documents("What is a LangGraph?")

[Document(metadata={'language': 'en', 'source': 'https://blog.langchain.dev/langgraph/', 'start_index': 3, 'title': 'LangGraph'}, page_content='LangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAll Posts\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024'),
 Document(metadata={'language': 'en', 'source': 'https://blog.langchain.dev/langgraph/', 'start_index': 3, 'title': 'LangGraph'}, page_content='LangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAll Posts\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\

In [46]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
    which might reference context in the chat history, formulate a standalone question \
    which can be understood without the chat history. Do NOT answer the question, \
    just reformulate it if needed and otherwise return it as is."""

In [47]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [48]:
from langchain_core.output_parsers import StrOutputParser

contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [49]:
from langchain_core.messages import AIMessage, HumanMessage

contextualize_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content = "What does LLM stand for?"),
            AIMessage(content = "Large language model")
        ],
        "question": "What is meant by large?",
    }
)

'LLM stands for Large Language Model.'

In [50]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you dont know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context} """


In [51]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [52]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [53]:
def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]

In [54]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    RunnablePassthrough.assign(
        context = contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | llm
)

In [55]:
chat_history = []

question = "What is LangGraph?"
ai_msg = rag_chain.invoke(
    {
        "question": question,
        "chat_history": chat_history
    }
)

In [56]:
ai_msg

'LangGraph is a module built on top of LangChain that enables the creation of cyclical graphs, often needed for agent runtimes. It provides an easy way to create these graphs and allows users to leverage existing LangChain agents within them.'

In [57]:
chat_history.extend(
    [
        HumanMessage(content = question), ai_msg
    ]
)

In [61]:
second_question = "What is it used for?"

rag_chain.invoke(
    {
        "question": second_question,
        "chat_history": chat_history
    }
)

'LangGraph is used to enable the creation of cyclical graphs in agent runtimes, particularly in Chat Agent Executors. This enables more custom and powerful agent runtimes by allowing developers to integrate tools into these cycles in a controlled manner.'